In [2]:
pip install python_docx

Note: you may need to restart the kernel to use updated packages.


In [72]:
import glob
import pandas as pd
import numpy as np
import pathlib
from docx import Document

In [62]:
def is_empty(e):
    if e is None:
        return True
    if isinstance(e, str) and not e:
        return True
    if isinstance(e, float) and np.isnan(e):
        return True
    if hasattr(e, '__len__') and len(e) == 0:
        return True
    return False

def is_string(e):
    if isinstance(e, str):
        if len(e.replace("\t", "")
               .replace("\n", "")
               .replace(" ", "")
              ) > 0:
            return True
    return False

def is_number(e):
    if isinstance(e, float) and not np.isnan(e):
        return True
    if isinstance(e, str) and e:
        return ((e
                .replace("\t", "")
                .replace("\n", "")
                .replace(" ", "")
                .replace("-", "")
                .replace("%", "")
                .replace(",", ".")
                .replace(".", "")
                .isdigit()
               ) or
                0 == len(e
                .replace("\t", "")
                .replace("\n", "")
                .replace(" ", "")
                .replace("-", "")
                .replace("%", "")
                .replace(",", "")
                .replace(".", "")))
    return False

def identification_mask(pdf):
    def substitute(e):
        if is_empty(e):
            return None
        if is_number(e):
            return "n"
        if is_string(e):
            return "s"
    return pdf.applymap(substitute)

In [63]:
def identify_number_area(c):
    ic = identification_mask(c)
    dc = ic.copy()
    
    max_i, max_j = ic.shape
    index_sum = float('inf')
    for i in range(ic.shape[0]):
        for j in range(ic.shape[1]):
            if 0 == (ic.iloc[i:, j:] == "s").sum().sum() and i + j < index_sum:
                index_sum = i + j
                max_i, max_j = i, j  

    return max_i, max_j

In [64]:
def squeeze_headers(c, number_area):
    
    def apply_transform(s):
        return s.apply(str).apply(lambda x: "" if x == "NaN" else x)
    
    sc = c.applymap(lambda x: "NaN" if isinstance(x, float) and np.isnan(x) else x)
    ie, je = number_area
    
    l1 = None
    for i in range(0, ie):
        if l1 is None:
            l1 = apply_transform(sc.iloc[0, :])
        else:
            l1 = l1.str.cat(apply_transform(sc.iloc[i]), sep=' ')
    if l1 is not None:
        sc = pd.concat([l1.to_frame().T, sc.drop(axis=0, index=range(0, ie))])
    
    l2 = None
    for i in range(0, je):
        if l2 is None:
            l2 = apply_transform(sc.iloc[:, 0])
        else:
            l2 = l2.str.cat(apply_transform(sc.iloc[:, i]), sep=' ')
    if l2 is not None:
        sc = pd.concat([l2.to_frame(), sc.drop(axis=1, columns=range(0, je))], axis=1)
    
    if l1 is not None:
        sc.iloc[0, :] = [f"*{i}" if x.strip() == "" else x for i, x in enumerate(sc.iloc[0, :])]
        sc.columns = sc.iloc[0, :]
        sc = sc.drop(axis=0, index=[0])
        
    sc = sc.set_index(sc.columns[0])
    
    return sc

def squeeze_columns(sc):
    columns_to_drop = []
    for c1, c2 in zip(sc.columns, sc.columns[1:]):
        all_is_nan = (sc[c1] == "NaN").all()
        if not isinstance(all_is_nan, pd.Series) and all_is_nan and isinstance(c2, str) and c2[0] == "*":
            sc[c1] = sc[c2]
            columns_to_drop.append(c2)
            
    return sc.drop(columns=columns_to_drop)

In [65]:
def import_from_docx(path):
    f = pathlib.Path(path)
    
    document = Document(f)
    
    tables = []
    for i, t in enumerate(document.tables):
        table = []
        for col in t.rows:
            table.append([])
            for c in col.cells:
                table[-1].append(c.text if c.text != "" else float('nan'))

        tables.append(pd.DataFrame(table))
    
    return tables

In [66]:
def import_from_csv(path):
    for i, f in enumerate(glob.glob(path + "/*.csv", recursive=True)):
        print("parsing", i, f)
        f = pathlib.Path(f)
        c = pd.read_csv(f, sep=',', thousands=' ', header=None, dtype=object, encoding='UTF-8')

In [69]:
def process_tables(tables, file_name):
    output_filename = "../data/nice/" + file_name + ".xlsx"
    writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')
    
    total_numbers = 0
    
    for i, table in enumerate(tables):
        c = table
#         display(c)
        ic = identification_mask(c)
#         display(ic)
        na = identify_number_area(c)
#         print(na)
        sc = squeeze_headers(c, na)
    #     display(sc)
        final = squeeze_columns(sc)
        
        try:
            final = final.applymap(lambda x: x.replace(" ", "").replace("%", "").replace(",", ".")).astype('float')
            total_numbers += final[final > 0].count().sum()
        except ValueError:
            pass

#         display(final.iloc[:10,:])
    
        final.to_excel(writer, sheet_name=f"Sheet{i}")
    writer.close()

    print("total_numbers", total_numbers)

In [70]:
docx_path = "C:\/Users\/mstransky003\/projects\/sfcr_reports\/data\/res\/BG/AXA SFCR 2020.docx"

tables = import_from_docx(docx_path)
process_tables(tables, "AXA SFCR 2020")

total_numbers 0


In [71]:
total_numbers = 0
for i, f in enumerate(glob.glob("../data/res/**/*.csv", recursive=True)):
    print("parsing", i, f)
    f = pathlib.Path(f)
    c = pd.read_csv(f, sep=',', thousands=' ', header=None, dtype=object, encoding='UTF-8')
#     display(c)
    ic = identification_mask(c)
#     display(ic)
    na = identify_number_area(c)
    print(na)
    sc = squeeze_headers(c, na)
#     display(sc)
    final = squeeze_columns(sc)
    
    try:
        final = final.applymap(lambda x: x.replace(" ", "").replace("%", "").replace(",", ".")).astype('float')
        total_numbers += final[final > 0].count().sum()
    except ValueError:
        pass
        
    display(final.iloc[:10,:])

    output_filename = "../data/nice/" + "".join(f.name.split(".")[:-1]) + ".xlsx"
    final.to_excel(output_filename)
    
    
print("total_numbers", total_numbers)

total_numbers 0


In [74]:
def to_float(s):
    try:
        return float(s)
    except ValueError:
        return s


def dataframe_to_couples(df):
    dfd = df.to_dict()
    lst = []
    for k, v in dfd.items():
        kl = k.lower().split(" ")
        for ki, vi in v.items():
            kil = ki.lower().split(" ")
            lst.append((set(kl).union(set(kil)), to_float(vi), k + " " + ki))
    return lst 


def mapping_table_to_words(df):
    transform = lambda x: [x[c].lower().split(" ") if isinstance(x[c], str) else "" for c in x.index]
    df["words"] = df.apply(transform, axis=1)
    df["words"] = df.apply(lambda x: set(b for a in x["words"] for b in a), axis=1)
    return df


def mapping_match(words, mapped):
    return len(mapped.intersection(set(words)))
    

df = pd.read_excel(f"../data/mapping_table.xlsx", header=0)
words = mapping_table_to_words(df)
words["best_match_val"] = -1
words["match"] = ""
words["mapping"] = ""
# display(words)
    
for i, f in enumerate(glob.glob("../data/nice/*.xlsx", recursive=True)):
    f = pathlib.Path(f)    
    df = pd.read_excel(f, header=0)
    df = df.set_index(df.columns[0])
#     display(df)
    couples = dataframe_to_couples(df)
    for couple in couples:
        words["mapping_match"] = words.apply(lambda x: mapping_match(x["words"], couple[0]), axis=1)
        best_idx = words["mapping_match"].idxmax()
        best_val = words["mapping_match"].max()
        if best_val > words.loc[best_idx, "best_match_val"] and best_val >= 2:
            words.at[best_idx, "match"] = couple[2]
            words.at[best_idx, "mapping"] = couple[1]
            words.at[best_idx, "best_match_val"] = best_val

display(words)
del words["words"]
del words["mapping_match"]
words.to_excel("../data/mapping_result.xlsx")

,S-code,C-code,R-code,Common,Text,Custom,words,best_match_val,match,mapping,mapping_match
0,QRT_S.02.01,C0010,R0040,NaN,Deferred tax assets,NaN,"{deferred, qrt_s.02.01, c0010, r0040, assets, ...",-1,,,0
1,QRT_S.02.01,C0010,R0070,NaN,Investments (other than assets held for index-...,NaN,"{assets, index-linked, r0070, qrt_s.02.01, hel...",-1,,,1
2,QRT_S.02.01,C0010,R0080,NaN,Property (other than for own use),NaN,"{own, qrt_s.02.01, property, (other, c0010, us...",-1,,,0
3,QRT_S.02.01,C0010,R0090,NaN,"Holdings in related undertakings, including pa...",NaN,"{including, in, related, participations, holdi...",-1,,,0
4,QRT_S.02.01,C0010,R0100,NaN,Equities,NaN,"{equities, r0100, c0010, qrt_s.02.01}",-1,,,0
...,...,...,...,...,...,...,...,...,...,...,...
183,QRT_S.05.01.02,C0240,R1800,Changes in other technical provisions,Other life insurance,life,"{in, other, life, insurance, changes, provisio...",-1,,,1
184,QRT_S.05.01.02,C0250,R1800,Changes in other technical provisions,Annuities stemming from non-life insurance con...,life,"{, life, to, r1800, and, stemming, other, from...",10,Line of Business for: non-life insurance and r...,NaN,4
185,QRT_S.05.01.02,C0260,R1800,Changes in other technical provisions,Annuities stemming from non-life insurance con...,life,"{life, to, r1800, and, c0260, stemming, other,...",-1,,,4
186,QRT_S.05.01.02,C0270,R1800,Changes in other technical provisions,Health reinsurance,life,"{in, other, life, health, changes, provisions,...",-1,,,1
